# Sklearn Tutorial from EDS
## Predicting Wine Quality
#### https://elitedatascience.com/python-machine-learning-tutorial-scikit-learn

## Imports

In [34]:
import numpy as np # numerical computation
import pandas as pd # handle numerical matrices

# import sampling helper
from sklearn.model_selection import train_test_split

# import preprocessing modules
from sklearn import preprocessing

In [35]:
# import random forest model
from sklearn.ensemble import RandomForestRegressor

# import cross validation pipeline
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

# import evaluation metrics
from sklearn.metrics import mean_squared_error, r2_score

# import module for saving sklearn models
from sklearn.externals import joblib
import joblib

### Import Dataset

In [36]:
dataset_url = 'http://mlr.cs.umass.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
data = pd.read_csv(dataset_url, sep=';')

In [37]:
print(data.shape)
data.head()

(1599, 12)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


### Summary Statistics

In [38]:
data.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [39]:
print("12 FEATURES:")
for x in data.columns: print(x)

12 FEATURES:
fixed acidity
volatile acidity
citric acid
residual sugar
chlorides
free sulfur dioxide
total sulfur dioxide
density
pH
sulphates
alcohol
quality


### Separate Data into target/train features and test/train datasets

In [40]:
y = data.quality
X = data.drop('quality', axis=1)

#### Set aside 20% of data as a testset for evaluating our model. Set arbitrary random seed so we can reproduce results.

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=123,
                                                    stratify=y)

### Use a Transformer to add a scaling preprocessing step.

<ol>
    <li>Fit transformer on training set, saving means and SDs.</li>
    <li>Apply transformer to training set, scaling training data using means and SDs.</li>
    <li>Apply transformer to test set, scaling test data using the same means and SDs.</li>
</ol>

In [42]:
# Fit Transformer API on training data
scaler = preprocessing.StandardScaler().fit(X_train)

In [43]:
# Apply transformer to training data
X_train_scaled = scaler.transform(X_train)

# All 11 features centered at mean = 0.
print(X_train_scaled.mean(axis=0))

# All 11 features with SD = 1, unit variance.
print(X_train_scaled.std(axis=0))

[ 1.16664562e-16 -3.05550043e-17 -8.47206937e-17 -2.22218213e-17
  2.22218213e-17 -6.38877362e-17 -4.16659149e-18 -2.54439854e-15
 -8.70817622e-16 -4.08325966e-16 -1.17220107e-15]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [44]:
# Apply transformer to the test data
X_test_scaled = scaler.transform(X_test)

# All 11 features not perfectly centered at 0 with unit variance
print(X_test_scaled.mean(axis=0))
print(X_test_scaled.std(axis=0))

[ 0.02776704  0.02592492 -0.03078587 -0.03137977 -0.00471876 -0.04413827
 -0.02414174 -0.00293273 -0.00467444 -0.10894663  0.01043391]
[1.02160495 1.00135689 0.97456598 0.91099054 0.86716698 0.94193125
 1.03673213 1.03145119 0.95734849 0.83829505 1.0286218 ]


#### Pipeline with preprocessing (scaling) and model (random forest)

No need to manually fit Transformer API.

In [45]:
# Create modeling pipeline
# First transforms the data using StandardScaler()
# Then fits a model using RandomForestRegressor()
pipeline = make_pipeline(preprocessing.StandardScaler(), 
                         RandomForestRegressor(n_estimators=100))

### Tuning Hyperparameters

<p><strong><em>Model Parameters:</em></strong> parameters that can be learned directly from the data, e.g. regression coefficients.</p>

<p><strong><em>Hyperparameters:</em></strong> parameters that express "higher-level" structural information about the model, typically set before the training data.</p>

#### Example: RandomForestRegressor

<p>Within each decision tree, the computer can empirically decide where to create branches based on either mean-squared-error (MSE) or mean-absolute-error (MAE). Therefore, the actual branch locations are <strong>model parameters</strong>.</p>

<p>However, the algorithm does not know which of the two criteria, MSE or MAE, that it should use. The algorithm also cannot decide how many trees to include in the forest. These are examples of <strong>hyperparameters</strong> that the user must set.</p>

#### List Tunable Hyperparameters

In [46]:
print(pipeline.get_params())

{'memory': None, 'steps': [('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('randomforestregressor', RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False))], 'verbose': False, 'standardscaler': StandardScaler(copy=True, with_mean=True, with_std=True), 'randomforestregressor': RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
           

#### Declare Hyperparameters to Tune

In [47]:
hyperparameters = { 'randomforestregressor__max_features' : ['auto', 'sqrt', 'log2'],
                  'randomforestregressor__max_depth': [None, 5, 3, 1]}

### Cross Validation

<p>Cross-validation is a process for reliably estimating the performance of <strong>a method</strong> for building a model by training and evaluating your model multiple times using the same method. This helps maximize model performance while <em>reducing the chance of overfitting</em>.</p>

<p>Practically, that "method" is simply a set of hyperparameters in this context.</p>

#### Steps for Cross Validation

<ol>
    <li>Split your data into k equal parts, or "folds" (typically k=10).</li>
    <li>Train your model on k-1 folds (e.g. the first 9 folds).</li>
    <li>Evaluate it on the remaining "hold-out" fold (e.g. the 10th fold).</li>
    <li>Perform steps (2) and (3) k times, each time holding out a different fold.</li>
    <li>Aggregate the performance across all k folds. This is your performance metric.</li>
</ol>

<img src="https://elitedatascience.com/wp-content/uploads/2016/12/K-fold_cross_validation_EN.jpg"></img>

#### Why Cross Validation?
<p>
Let's say you want to train a random forest regressor. One of the hyperparameters you must tune is the maximum depth allowed for each decision tree in your forest.</p>

<p>How can you decide?</p>

<p>That's where cross-validation comes in. Using only your training set, you can use CV to evaluate different hyperparameters and estimate their effectiveness.</p>

<p>This allows you to keep your test set "untainted" and save it for a true hold-out evaluation when you're finally ready to select a model.</p>

<p>For example, you can use CV to tune a random forest model, a linear regression model, and a k-nearest neighbors model, using only the training set. Then, you still have the untainted test set to make your final selection between the model families!</p>

#### Cross Validation Pipeline

<p>The best practice when performing CV is to include your data preprocessing steps inside the cross-validation loop. This prevents accidentally tainting your training folds with influential data from your test fold.</p>
<p>Here's how the CV pipeline looks after including preprocessing steps:
<ol>
    <li>Split your data into k equal parts, or "folds" (typically k=10).</li>
    <li><strong>Preprocess k-1 training folds.</strong></li>
    <li>Train your model on the same k-1 folds.</li>
    <li><strong>Preprocess the hold-out fold using the same transformations from step (2).</strong></li>
    <li>Evaluate your model on the same hold-out fold.</li>
    <li>Perform steps <strong>(2) - (5)</strong> k times, each time holding out a different fold.</li>
    <li>Aggregate the performance across all k folds. This is your performance metric.</li>
</ol>
</p>

In [48]:
# Sklearn cross-validation with pipeline
clf = GridSearchCV(pipeline, hyperparameters, cv=10)
 
# Fit and tune model
clf.fit(X_train, y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('standardscaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('randomforestregressor',
                                        RandomForestRegressor(bootstrap=True,
                                                              criterion='mse',
                                                              max_depth=None,
                                                              max_features='auto',
                                                              max_leaf_nodes=None,
                                                              min_impurity_decrease=0.0,
                                                              min_impurity_split

<p><strong>GridSearchCV</strong> essentially performs cross-validation across the entire "grid" (all possible permutations) of hyperparameters.</p>

<p>It takes in your model (in this case, we're using a model pipeline), the hyperparameters you want to tune, and the number of folds to create.</p>

In [50]:
# Print the best parameters found by GridSearchCV
print(clf.best_params_)

{'randomforestregressor__max_depth': None, 'randomforestregressor__max_features': 'log2'}


### Refitting on the entire training set

<p>After tuning the hyperparameters appropriately using cross-validation, refitting the model on the entire training set can generally get a small performance improvement.</p>

<p>Conveniently, <strong>GridSearchCV</strong> from sklearn will automatically refit the model with the best set of hyperparameters using the entire training set, <strong>on by default</strong>.</p>

In [51]:
# Confirm model will be retrained on entire training set.
print(clf.refit)

True


### Evaluate model pipeline on test data.

In [56]:
# Predicting a new set of data.
y_pred = clf.predict(X_test)

In [57]:
# Evaluate model performance
print("r2:", r2_score(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))

r2: 0.4632517706882985
MSE: 0.34634937499999996


### Improving Model Performance

<ol>
    <li>Try other regression model families (e.g. regularized regression, boosted trees, etc.).</li>
    <li>Collect more data if it's cheap to do so.</li>
    <li>Engineer smarter features after spending more time on exploratory analysis.</li>
    <li>Speak to a domain expert to get more context (...this is a good excuse to go wine tasting!).</li>
</ol>

### Saving/Loading model for future use.


<p><strong>Save model</strong> to .pkl file: <code>joblib.dump(clf, 'rf_regressor.pkl')</code></p>

<p><strong>Load model</strong> from .pkl file: <code>clf2 = joblib.load('rf_regressor.pkl')</code></p>
<p>Then, predict data set using loaded model: <code>clf2.predict(X_test)</code></p>